# Figure 4 - reconstruction algorithm explained

This notebook shows the different steps involved in the axon reconstruction algorithm.

To run this notebook, you first need to run `axon_velocity/simulations/simulation_notebooks/simulate_cell_1.ipynb` to generate the simulated data.

In [ ]:
import numpy as np
import matplotlib.pylab as plt
import MEAutility as mu
from scipy.signal import resample_poly
from scipy.stats import kurtosis, linregress
from matplotlib import gridspec
from scipy import io
import numpy as np
import networkx as nx
from pathlib import Path
import sys
from pprint import pprint

%matplotlib widget
#%matplotlib notebook

from axon_velocity import *
from axon_velocity.models import load_cell
from axon_velocity.evaluation import *

In [ ]:
save_fig = True
fig_folder =  Path('figures') / "figure4"
fig_folder.mkdir(exist_ok=True, parents=True)

### Define algorithm params

In [ ]:
params = get_default_graph_velocity_params()
pprint(params)
# change params
params['detect_threshold'] = 0.01
params['kurt_threshold'] = 0.5
params['peak_std_threshold'] = None
params['init_delay'] = 0.1
params['upsample'] = 5
params['neighbor_radius'] = 50
params["r2_threshold"] = 0.9
params["max_distance_for_edge"] = 100
params["max_distance_to_init"] = 300
params["max_peak_latency_for_splitting"] = 0.5
params["min_path_points"] = 8

# Cell 1

In [ ]:
morphology_folder = Path('..') / 'simulations' / 'neuromorpho' / 'allen_cell_types'
data_folder = Path('..') / 'simulations' / 'simulated_data' / 'allen' 
zspan = 0

In [ ]:
morphology_files_dict = {"cell1": morphology_folder / 'H16-06-008-01-20-04_561096006_m.CNG.swc', 
                         "cell2": morphology_folder / 'H16-06-004-01-04-01_538906745_m.CNG.swc',
                         "cell3": morphology_folder / 'H16-03-006-01-04-03_563818992_m.CNG.swc', 
                         "cell4": morphology_folder / 'H17-06-006-11-08-02_606834771_m.CNG.swc'}
cell_folders_dict = {"cell1": data_folder / f'allen0_planar_{zspan}um' , 
                     "cell2": data_folder / f'allen1_planar_{zspan}um' ,
                     "cell3": data_folder / f'allen2_planar_{zspan}um' , 
                     "cell4": data_folder / f'allen3_planar_{zspan}um' }

In [ ]:
cell_name = "cell1"
cell_folder = cell_folders_dict[cell_name]
morphology_file = morphology_files_dict[cell_name]

In [ ]:
cell_path = [p for p in cell_folder.iterdir() if p.suffix == '.pkl'][0]
locs_path = [p for p in cell_folder.iterdir() if 'locations' in p.name][0]
template_path = [p for p in cell_folder.iterdir() if 'template' in p.name][0]

In [ ]:
cell, sections = load_cell(cell_path)
locations = np.load(locs_path)
template = np.load(template_path)

In [ ]:
fs = 1 / cell.dt * 1000

In [ ]:
gtr = GraphAxonTracking(template, locations, fs, verbose=True, **params)

In [ ]:
gtr.select_channels()

In [ ]:
gtr.build_graph()

In [ ]:
gtr.find_paths()

In [ ]:
fig_graph = plt.figure(figsize=(10, 7))
fig_graph = gtr.plot_graph(node_search_labels=False, fig=fig_graph, cmap_nodes="viridis", cmap_edges="YlGn")

In [ ]:
gtr._verbose = 2

In [ ]:
gtr.clean_paths(remove_outliers=True)

In [ ]:
fpaths_raw, axpaths_raw = plt.subplots(figsize=(7, 10))
axpaths_raw = gtr.plot_raw_branches(cmap="tab20", plot_bp=True, plot_neighbors=True, plot_full_template=True,
                                    ax=axpaths_raw)
axpaths_raw.legend(fontsize=12)

In [ ]:
gtr._branching_points

In [ ]:
fvel, ax_vel = plt.subplots(figsize=(9, 15))
ax_vel.axis("off")
fvel = gtr.plot_velocities(cmap="tab20", plot_outliers=True, fig=fvel, markersize=12, markersize_out=18, fs=20)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)

In [ ]:
if save_fig:
    fig_graph.suptitle('')
    fig_graph.savefig(fig_folder / 'panelAB_graph.png', dpi=600)
    fpaths_raw.suptitle('')
    fpaths_raw.savefig(fig_folder / 'panelC_rawpaths.png', dpi=600)
    fvel.suptitle('')
    fvel.savefig(fig_folder / 'panelC_velocities.svg')